# Analysis of tweet text

In [1]:
import pymongo
import re
import pprint
import logging

In [2]:
# Logging functionality
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logging.debug('Start of program')

2018-05-08 22:04:23,682 - DEBUG - Start of program


In [3]:
# Get the tweets stored in MongoDB
client = pymongo.MongoClient()
tweets = client['twitter']['tweets'].find()
latest_tweet = tweets[22]

In [4]:
def remove_from_word_list(original_list, list_subset):
    for each in list_subset:
        idx = original_list.index(each)
        del original_list[idx]
    return original_list

In [5]:
def print_important_content(tweet):
    print('By {} on {}\n'.format(tweet['user']['name'], tweet['created_at']))
    print('"{}"\n'.format(tweet['full_text']))
    pprint.pprint(tweet['entities'])
    
print_important_content(latest_tweet)

By Alex Tabarrok on Mon Apr 30 18:49:54 +0000 2018

"RT @JustinSandefur: Super useful. Wonky econometrics post, but huge implications for lots of stuff you (think you) know about the effect of…"

{'hashtags': [],
 'symbols': [],
 'urls': [],
 'user_mentions': [{'id': 95652758,
                    'id_str': '95652758',
                    'indices': [3, 18],
                    'name': 'Justin Sandefur',
                    'screen_name': 'JustinSandefur'}]}


In [6]:
def hashtag(tweet):
    hashtag_list = []
    for tag in tweet['hashtags']:
        hashtag_list.append('#' + tag['text'])
    return hashtag_list

hashtags = hashtag(latest_tweet['entities'])
hashtags

[]

In [7]:
latest_tweet

{'_id': ObjectId('5ae766611c398f0d01baa6ea'),
 'created_at': 'Mon Apr 30 18:49:54 +0000 2018',
 'id': 991026874959695872,
 'id_str': '991026874959695872',
 'full_text': 'RT @JustinSandefur: Super useful. Wonky econometrics post, but huge implications for lots of stuff you (think you) know about the effect of…',
 'truncated': False,
 'display_text_range': [0, 140],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'JustinSandefur',
    'name': 'Justin Sandefur',
    'id': 95652758,
    'id_str': '95652758',
    'indices': [3, 18]}],
  'urls': []},
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 287309941,
  'id_str': '287309941',
  'name': 'Alex Tabarrok',
  'screen_name': 'ATabarrok',
  'location': '',
  'description': '',
 

In [8]:
# Break the text content apart for better formatting.
words = latest_tweet['full_text'].split(' ')

In [9]:
# Display the broken down list of each 'word'.
words

['RT',
 '@JustinSandefur:',
 'Super',
 'useful.',
 'Wonky',
 'econometrics',
 'post,',
 'but',
 'huge',
 'implications',
 'for',
 'lots',
 'of',
 'stuff',
 'you',
 '(think',
 'you)',
 'know',
 'about',
 'the',
 'effect',
 'of…']

In [10]:
words = remove_from_word_list(words, hashtags)
words

['RT',
 '@JustinSandefur:',
 'Super',
 'useful.',
 'Wonky',
 'econometrics',
 'post,',
 'but',
 'huge',
 'implications',
 'for',
 'lots',
 'of',
 'stuff',
 'you',
 '(think',
 'you)',
 'know',
 'about',
 'the',
 'effect',
 'of…']

In [11]:
# Find any urls in the list of words.
# Located better regex for url pattern matching - http://www.noah.org/wiki/RegEx_Python
def url_locator(word_list):
    urls = []
    for word in word_list:
        url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', word)
        if url:
            urls.append(url[0])
    return urls
urls = url_locator(words)
urls

[]

In [12]:
words = remove_from_word_list(words, urls)
words

['RT',
 '@JustinSandefur:',
 'Super',
 'useful.',
 'Wonky',
 'econometrics',
 'post,',
 'but',
 'huge',
 'implications',
 'for',
 'lots',
 'of',
 'stuff',
 'you',
 '(think',
 'you)',
 'know',
 'about',
 'the',
 'effect',
 'of…']

In [13]:
# remove @mentions
def detect_user_handles(word_list):
    user_handle = []
    for word in word_list:
        if word.startswith('@'):
            user_handle.append(word)
    return user_handle

users = detect_user_handles(words)
users

['@JustinSandefur:']

In [14]:
words = remove_from_word_list(words, users)
words

['RT',
 'Super',
 'useful.',
 'Wonky',
 'econometrics',
 'post,',
 'but',
 'huge',
 'implications',
 'for',
 'lots',
 'of',
 'stuff',
 'you',
 '(think',
 'you)',
 'know',
 'about',
 'the',
 'effect',
 'of…']

In [15]:
# def remove_extra_characters(word_list):
def strip_nonalphanumeric_characters(word_list):
    clean_list = []
    regex = re.compile('[^a-zA-Z0-9]')
    for word in word_list:
        word = regex.sub('', word)
        if word != '':
            clean_list.append(word)
    return clean_list
clean_words = strip_nonalphanumeric_characters(words)
clean_words

['RT',
 'Super',
 'useful',
 'Wonky',
 'econometrics',
 'post',
 'but',
 'huge',
 'implications',
 'for',
 'lots',
 'of',
 'stuff',
 'you',
 'think',
 'you',
 'know',
 'about',
 'the',
 'effect',
 'of']

In [16]:
list(set(words) - set(clean_words))

['useful.', 'post,', 'of…', 'you)', '(think']

In [17]:
logging.debug('End of program')

2018-05-08 22:04:23,803 - DEBUG - End of program
